In [1]:
import os
os.chdir('../')
%pwd

'h:\\Personal Projects\\Indian-Housing-Prices-Backend'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str  
    verbose: int
    random_seed: int
    target_column: str

In [3]:
from houseProject.constants import *
from houseProject.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            verbose=params.verbose,
            random_seed=params.random_seed,
            target_column=schema.name
        )

        return model_trainer_config

In [5]:
import pandas as pd
import os
from houseProject import logger
from catboost import CatBoostRegressor
import joblib

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Initialize RandomForestRegressor
        model = CatBoostRegressor(
            verbose=self.config.verbose,
            random_seed=self.config.random_seed,
        )

        # Train the model
        model.fit(train_x, train_y)

        # Save the model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-05-19 20:36:51,251: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 20:36:51,258: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 20:36:51,262: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-19 20:36:51,265: INFO: common: created directory at: artifacts]
[2025-05-19 20:36:51,266: INFO: common: created directory at: artifacts/model_trainer]
